Test of Variance algorithm performance, adapted from Rudy's implementation in:
- https://colab.research.google.com/drive/1JbvnecwCdaKpmxhugiLszQ3UXiIQ-dQ0?usp=sharing#scrollTo=BbpIleePIUaN

And described more extensively in:
- https://www.overleaf.com/project/672207ce18cc0a8388a1a7f8

In [1]:
# basic imports
import sys
import os
import numpy as np
import torch
import pandas as pd
import csv
import math
from scipy.stats import norm
from scipy import integrate
from typing import List, Tuple

In [2]:
# custom imports
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from src.utils import load_config
import src.graphs_generation as graphs_gen

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\danie\Desktop\Virtual Envs\ML\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Variance test definition

In [3]:
def variance_algo(graph, threshold):
  # computing empirical degree of each node
  degrees = graph.sum(dim=1)  # or dim=0, since the matrix is symmetric
  # computing the mean degree
  mean_degree = degrees.mean()
  # computing the empirical variance
  graph_variance = ((degrees - mean_degree) ** 2).mean()

  # comparing the variance to the threshold to determine if the graph has a planted clique
  if graph_variance > threshold:
    return 1
  else:
    return 0


# Test function (adapted)

In [4]:
# TEST FUNCTION
# read configuration file:
config = load_config(
    os.path.join("Variance-algo_test_config.yml")
)  # CHANGE THIS TO PERFORM DIFFERENT EXPERIMENTS

# looping over the different graph sizes in the experiment:
for graph_size in config["graph_size_values"]:

    # Create empty dictionaries for storing testing results:
    fraction_correct_results = {}  # Fraction correct for each clique size
    metrics_results = {}  # Metrics dictionary

    # Calculate max clique size (proportion of graph size):
    max_clique_size = int(
        config["testing_parameters"]["max_clique_size_proportion_test"] * graph_size
    )

    # Calculate array of clique sizes for all test curriculum
    # NOTE: if max clique size is smaller than the the number of test levels, use max clique size as the number of test levels
    clique_sizes = np.linspace(
        max_clique_size,
        2,  # to avoid division by zero in variance calculation
        num=min(max_clique_size, config["testing_parameters"]["clique_testing_levels"]),
    ).astype(int)
    
    # Metrics initialization (local to each GPU)
    TP, FP, TN, FN = 0, 0, 0, 0  

    # Loop for decreasing clique sizes
    for current_clique_size in clique_sizes:

        # Computing decision threshold for current N/K combination
        N = graph_size*(graph_size-1)/2
        q = current_clique_size*(current_clique_size-1)/(graph_size*(graph_size-1))
        p_0 = 0.5
        p_1 = 1 - 1/(1-q)*(1-p_0)
        variance_0 = p_0 * (1 - p_0) * N
        variance_1 = p_1 * (1 - p_0) * N
        z = q / (1 - q) * (1 - p_0) / p_0
        threshold = -variance_1 * np.log(1 - z) / z

        # Initialize fraction correct list, updated at each test iteration
        fraction_correct_list = []

        # Loop for testing iterations:
        for test_iter in range(config["testing_parameters"]["test_iterations"]):

            # Generate clique size value of each graph in the current batch
            clique_size_array_test = graphs_gen.generate_batch_clique_sizes(
                np.array([current_clique_size]),
                config["testing_parameters"]["num_test"],
            )

            # Generate validation graphs
            test = graphs_gen.generate_batch(
                config["testing_parameters"]["num_test"],
                graph_size,
                clique_size_array_test,
                config["p_correction_type"],
                False,
            )
            
            hard_output = torch.zeros([config["testing_parameters"]["num_test"]])
            
            for graph_index, graph in enumerate(test[0]):
                Variance_algo_output = variance_algo(graph.squeeze(), threshold)
                hard_output[graph_index] = Variance_algo_output
            
            # transforming hard_output and test_labels to torch tensors:
            hard_output = torch.tensor(hard_output, dtype=torch.float32)
            test_labels = torch.tensor(test[1], dtype=torch.float32)
            
            # Compute metrics
            TP += ((hard_output == 1) & (test_labels == 1)).sum().item()
            FP += ((hard_output == 1) & (test_labels == 0)).sum().item()
            TN += ((hard_output == 0) & (test_labels == 0)).sum().item()
            FN += ((hard_output == 0) & (test_labels == 1)).sum().item()

            # updating fraction correct list with the accuracy of the current test iteration:
            fraction_correct_list.append(
                (hard_output == test_labels).sum().item()
                / (1.0 * config["testing_parameters"]["num_test"])
            )

        # Updating dictionary after all test iterations for current clique size have been completed:
        fraction_correct_results[current_clique_size] = round(
            sum(fraction_correct_list) / len(fraction_correct_list), 2
        )

        # Printing the size of the clique just tested and the corresponding test accuracy:
        print(
            f"||| Completed testing for clique = {current_clique_size}. "
            f"Average fraction correct = {fraction_correct_results[current_clique_size]}"
        )
        print("|||===========================================================")

    # - notify completion of testing:
    print(f"| Finished testing Variance algo at N = {graph_size}.")

    # Computing metrics:
    precision = TP / (TP + FP + 1e-10)
    recall = TP / (TP + FN + 1e-10)
    F1 = 2 * (precision * recall) / (precision + recall + 1e-10)
    # AUC - ROC cannot be calculated (no soft outputs)
    # num_params has no meaning
    metrics_results = {
        "TP": TP,
        "FP": FP,
        "TN": TN,
        "FN": FN,
        "precision": precision,
        "recall": recall,
        "F1": F1,
        "AUC_ROC": np.nan,
        "total_params": np.nan,
    }

    # Saving accuracy results in .csv file:
    # - defining file name and path:
    file_path = os.path.join(
        os.getcwd(), "results", f"Variance-algo_N{graph_size}_fraction_correct.csv"
    )
    # - saving the dictionary as a .csv file:
    with open(file_path, "w") as file:
        writer = csv.writer(file)
        writer.writerow(["clique size", "fraction correct"])  # Add column labels
        for key, value in fraction_correct_results.items():
            writer.writerow([key, value])
    # Saving metrics results in .csv file:
    # - defining file name and path:
    file_path = os.path.join(
        os.getcwd(), "results", f"Variance-algo_N{graph_size}_metrics.csv"
    )
    # - saving the dictionary as a .csv file:
    pd.DataFrame([metrics_results]).to_csv(file_path, index=False)

    print(f"- Variance algo Results saved successfully for N = {graph_size}.")

Configuration file loaded successfully.


C:\Users\danie\AppData\Local\Temp\ipykernel_9560\3308773397.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hard_output = torch.tensor(hard_output, dtype=torch.float32)


||| Completed testing for clique = 840. Average fraction correct = 1.0
|||===========================================================
||| Completed testing for clique = 831. Average fraction correct = 1.0
|||===========================================================
||| Completed testing for clique = 823. Average fraction correct = 1.0
|||===========================================================
||| Completed testing for clique = 814. Average fraction correct = 1.0
|||===========================================================
||| Completed testing for clique = 806. Average fraction correct = 1.0
|||===========================================================
||| Completed testing for clique = 797. Average fraction correct = 1.0
|||===========================================================
||| Completed testing for clique = 789. Average fraction correct = 1.0
|||===========================================================
||| Completed testing for clique = 780. Average fraction corre